In [ ]:
import os
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [ ]:
CUR_DIR = os.getcwd()
PROJECT_DIR = os.path.dirname(CUR_DIR)
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')

In [ ]:
def getFiles(folderPath):
    files_name = []
    for root, dirs, files in os.walk(folderPath):
        for file in files:
            files_name.append(file)

    return files_name

In [ ]:
files_name = getFiles(RAW_DATA_DIR)

In [ ]:
FRANC_TEXT = os.path.join(RAW_DATA_DIR, files_name[0])

In [ ]:
device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open(FRANC_TEXT, 'r') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)
block_size = 64
batch_size = 128
num_embd = 500
num_head = 8
num_layer = 8
dropout = 0.2
learning_rate = 3e-6
max_iters = 10000

In [ ]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}

encode = lambda x: [str_to_int[c] for c in x]
decode = lambda x: ''.join([int_to_str[i] for i in x])

In [ ]:
# original 
text

In [ ]:
# encode
text_encode = encode(text)

In [ ]:
# decode
decode(text_encode)

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
data

In [ ]:
data.shape

In [ ]:
len(text)

In [ ]:
def get_batch(data, block_size=8, batch_size=4, gap=1):
    index = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+gap:i+block_size+gap] for i in index])
    return x.to(device), y.to(device)

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
train

In [ ]:
X, y = get_batch(train)

In [ ]:
X

In [ ]:
y

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
decode(X[0].tolist())

In [ ]:
decode(y[0].tolist())

In [ ]:
import torch.nn.functional as F

In [ ]:
n_embd = num_embd
n_layer = num_layer
n_head = num_head

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [ ]:
# model = GPTLanguageModel(vocab_size, num_embd, block_size, num_head, num_layer)
# m = model.to(device)

# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# gen_chars = decode(m.generate(context, max_new_token=500)[0].tolist())
# print(gen_chars)

In [ ]:
def train_model(model, num_epoch, optimizer, train_data, test_data):
    losses = {}

    model.train()
    for epoch in range(num_epoch):
        xb, yb = get_batch(train_data)

        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        #
        if epoch % 1000 == 0:
            # track train loss
            losses['train'] = loss.item()

            # evaluation mode
            model.eval()
            xb, yb = get_batch(test_data)
            logits, loss = model(xb, yb)
            losses['test'] = loss.item()

            print("Epoch %d : Train %.4f, Test %.4f" % (epoch, losses['train'], losses['test']))

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
train_model(model, max_iters, optimizer, train, test)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
gen_chars = decode(m.generate(context, 500)[0].tolist())
print(gen_chars)